In [1]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['lines.markeredgewidth'] = 0.5
matplotlib.rcParams['lines.markersize'] = 8 

In [2]:
import numpy as np
from numba import njit, prange

In [3]:
from ns2d import FiniteDifferenceDiscretizer, EulerIntegrator

In [ ]:
# Write the solve_poisson and update_velocity concrete functions in a MainSolver class

In [ ]:
# nx, ny = 41, 41
# dx, dy = 2.0 / (nx - 1), 2.0 / (ny - 1)
# dt = 0.001
# nu = 0.1
# num_steps = 500

# x = np.linspace(0, 2, nx)
# y = np.linspace(0, 2, ny)
# X, Y = np.meshgrid(x, y)

# spatial_discretizer = FiniteDifferenceDiscretizer()
# time_integrator = EulerIntegrator()
# solver = MainSolver(nx=nx, ny=ny, dx=dx, dy=dy, dt=dt, nu=nu,
#                     integrator=time_integrator, discrete_navier_stokes=spatial_discretizer)

# solver.run_simulation(num_steps=num_steps, initialize=True)

In [ ]:
# plt.figure(figsize=(11, 7))
# plt.contourf(X, Y, solver.p, alpha=0.5)
# plt.colorbar(label='Pressure')
# plt.contour(X, Y, solver.p, colors='black', linestyles='solid')
# plt.quiver(X[::2, ::2], Y[::2, ::2], solver.u[::2, ::2], solver.v[::2, ::2], color='r')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.tight_layout()
# plt.show()